Эфемериды — это таблицы положений небесных тел во времени. Для точного моделирования нужны координаты Земли и Марса в инерциальной системе отсчета (ICRF) на моменты ваших измерений.

Интегрируем в наш код SPICE (пакет программ NASA для рассчетов в области космической геометрии) - это готовая система, которая предоставит:
1. Векторы состояния Земли, Марса и Солнца 
2. Координаты наземных станций DSN с учетом вращения Земли
3. Преобразование систем координат (например, из инерциальной в систему, связанную с вращением Земли)
4. Преобразования времени (UTC -> TDB, которое используется в динамических уравнениях)

Загрузим файлы SPICE-ядер

In [33]:
import os
import requests
import zipfile
import tarfile
import gzip
import shutil
import subprocess
from pathlib import Path
from tqdm import tqdm

class SmartMGSSpiceDownloader:
    def __init__(self, base_dir="spice_data_smart", verbose=True, force_redownload=False):
        self.base_dir = Path(base_dir)
        self.verbose = verbose
        self.force_redownload = force_redownload
        self.ensure_directories()
    
    def ensure_directories(self):
        dirs = ['kernels/spk', 'kernels/lsk', 'kernels/pck', 'kernels/sclk', 
                'toolkits', 'tmp', 'checksums']
        for dir_name in dirs:
            (self.base_dir / dir_name).mkdir(parents=True, exist_ok=True)
    
    def is_file_exists_and_valid(self, filepath):
        """
        Проверка: файл существует и не является HTML-ошибкой.
        
        Args:
            filepath: Путь к файлу
        
        Returns:
            bool: True если файл существует и валиден
        """
        if not filepath.exists():
            if self.verbose:
                print(f"    Файл не существует: {filepath.name}")
            return False
        
        # Быстрая проверка, что это не HTML-ошибка (битая загрузка)
        try:
            if filepath.stat().st_size > 100:
                with open(filepath, 'rb') as f:
                    header = f.read(100)
                    # Проверка, что это не HTML/текстовая ошибка
                    if b'html' in header.lower() or b'error' in header.lower():
                        if self.verbose:
                            print(f"    Файл содержит HTML (ошибка загрузки): {filepath.name}")
                        return False
        except:
            pass
        
        if self.verbose:
            file_size = filepath.stat().st_size / 1024 / 1024
            print(f"    ✓ Файл уже существует: {filepath.name} ({file_size:.1f} MB)")
        return True
    
    def download_file_with_check(self, url, destination, description="Файл", skip_if_exists=True):
        """
        Загрузка с проверкой существования файла.
        
        Args:
            url: URL для скачивания
            destination: Путь для сохранения
            description: Описание файла
            skip_if_exists: Пропустить если файл уже существует
        
        Returns:
            bool: True если файл загружен или уже существует
        """
        # Создаем директорию если нужно
        destination.parent.mkdir(parents=True, exist_ok=True)
        
        # Проверяем, нужно ли скачивать
        if (skip_if_exists and not self.force_redownload and 
            self.is_file_exists_and_valid(destination)):
            return True
        
        if self.verbose:
            if destination.exists():
                print(f"    Перезаписываю файл: {destination.name}")
            else:
                print(f"    Загружаю новый файл: {destination.name}")
        
        try:
            headers = {'User-Agent': 'Mozilla/5.0'}
            
            # Поддержка докачки
            if destination.exists() and not self.force_redownload:
                file_size = destination.stat().st_size
                headers['Range'] = f'bytes={file_size}-'
                mode = 'ab'
                resume = True
            else:
                mode = 'wb'
                resume = False
            
            response = requests.get(url, headers=headers, stream=True, timeout=60)
            response.raise_for_status()
            
            # Получаем размер
            total_size = int(response.headers.get('content-length', 0))
            if resume:
                total_size += file_size
            
            # Скачивание с прогресс-баром
            with open(destination, mode) as f:
                with tqdm(
                    desc=f"{description:30}",
                    total=total_size,
                    unit='B',
                    unit_scale=True,
                    initial=destination.stat().st_size if resume else 0,
                    disable=not self.verbose
                ) as pbar:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                            pbar.update(len(chunk))
            
            # Базовая проверка скачанного файла
            if destination.exists() and destination.stat().st_size > 0:
                if self.verbose:
                    print(f"    ✓ Успешно: {destination.name}")
                return True
            else:
                print(f"    ✗ Файл не скачан или пустой: {destination.name}")
                return False
                
        except Exception as e:
            print(f"    ✗ Ошибка загрузки {url}: {e}")
            # Удаляем неполный файл
            if destination.exists():
                destination.unlink()
            return False
    
    def download_ephemeris_smart(self, version='de405'):
        """Загрузка эфемерид с проверкой."""
        spk_dir = self.base_dir / 'kernels' / 'spk'
        filename = f"{version}.bsp"
        dest_file = spk_dir / filename
        
        # Проверяем, есть ли уже файл
        if not self.force_redownload and self.is_file_exists_and_valid(dest_file):
            return True
        
        if self.verbose:
            print(f"\n{'='*60}")
            print(f"Загрузка эфемерид {version.upper()}")
            print(f"{'='*60}")
        
        # URL
        ephemeris_urls = {
            'de405': [
                'https://ssd.jpl.nasa.gov/ftp/eph/planets/bsp/de405.bsp',
                'https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de405.bsp'
            ],
            'de430': ['https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de430.bsp'],
            'de440': ['https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de440.bsp'],
            'de441': ['https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de441.bsp']
        }
        
        if version not in ephemeris_urls:
            print(f"Версия {version} не найдена. Доступно: {list(ephemeris_urls.keys())}")
            version = 'de405'
        
        urls = ephemeris_urls[version]
        success = False
        
        # Пробуем все доступные URL
        for url in urls:
            if self.verbose:
                print(f"  Пробую URL: {url}")
            
            success = self.download_file_with_check(
                url, dest_file, f"Эфемериды {version.upper()}"
            )
            
            if success:
                break
            else:
                if self.verbose:
                    print(f"  Не удалось загрузить с {url}")
        
        if success and self.verbose:
            file_size = dest_file.stat().st_size / 1024 / 1024
            print(f"Эфемериды {version.upper()} готовы: {file_size:.1f} MB")
        
        return success
    
    def download_spice_kernels_smart(self):
        """Умная загрузка базовых ядер SPICE - ОБНОВЛЕНО для DE405."""
        base_url = "https://naif.jpl.nasa.gov/pub/naif/generic_kernels/"
        
        # Для DE405 лучше использовать pck00010.tpc
        kernels = [
            ('lsk/naif0012.tls', self.base_dir / 'kernels' / 'lsk' / 'naif0012.tls'),
            ('pck/pck00010.tpc', self.base_dir / 'kernels' / 'pck' / 'pck00010.tpc'),
        ]
        
        if self.verbose:
            print(f"\nПроверка базовых ядер SPICE...")
        
        all_success = True
        for kernel_path, dest_path in kernels:
            url = base_url + kernel_path
            
            # Проверяем существование
            if not self.force_redownload and self.is_file_exists_and_valid(dest_path):
                continue
            
            success = self.download_file_with_check(
                url, dest_path, f"Ядро {Path(kernel_path).name}"
            )
            all_success = all_success and success
        
        return all_success
    
    def check_toolkit_extracted(self, platform_dir):
        """Проверяет, распакован ли уже toolkit."""
        if not platform_dir.exists():
            return False
        
        # Проверяем наличие ключевых файлов
        required_files = ['cspice/lib/cspice.a', 'cspice/include/SpiceUsr.h']
        for file in required_files:
            if not (platform_dir / file).exists():
                return False
        
        if self.verbose:
            print(f"    Toolkit уже распакован: {platform_dir.name}")
        return True
    
    def download_spice_toolkit_smart(self, platform='PC_Linux_GCC_64bit'):
        """Загрузка SPICE Toolkit."""
        tmp_dir = self.base_dir / 'tmp'
        toolkit_dir = self.base_dir / 'toolkits' / platform
        
        # Проверяем, не распакован ли уже toolkit
        if not self.force_redownload and self.check_toolkit_extracted(toolkit_dir):
            return True
        
        if self.verbose:
            print(f"\nОбработка SPICE Toolkit для {platform}...")
        
        toolkits = {
            'PC_Linux_GCC_64bit': {
                'url': 'https://naif.jpl.nasa.gov/pub/naif/toolkit/C/PC_Linux_GCC_64bit/packages/cspice.tar.Z',
                'format': 'tar.Z',
            },
            'PC_Windows_VisualC_64bit': {
                'url': 'https://naif.jpl.nasa.gov/pub/naif/toolkit/C/PC_Windows_VisualC_64bit/packages/cspice.zip',
                'format': 'zip',
            },
        }
        
        if platform not in toolkits:
            platform = 'PC_Linux_GCC_64bit'
        
        toolkit_info = toolkits[platform]
        url = toolkit_info['url']
        archive_name = Path(url).name
        archive_path = tmp_dir / archive_name
        
        # Скачиваем архив если нужно
        if not self.force_redownload and archive_path.exists() and archive_path.stat().st_size > 0:
            if self.verbose:
                print(f"    Архив уже существует: {archive_name}")
            # Не удаляем, просто пропускаем
            
        success = self.download_file_with_check(
            url, archive_path, f"Toolkit {platform}"
        )
        
        if not success:
            return False
        
        # Распаковка
        if self.verbose:
            print(f"    Распаковка архива...")
        
        try:
            if toolkit_info['format'] == 'zip':
                with zipfile.ZipFile(archive_path, 'r') as zip_ref:
                    zip_ref.extractall(toolkit_dir)
            elif toolkit_info['format'] == 'tar.Z':
                # Используем gunzip если доступен
                try:
                    subprocess.run(['gunzip', '-f', str(archive_path)], 
                                 check=True, capture_output=True)
                    tar_path = archive_path.with_suffix('')
                    with tarfile.open(tar_path, 'r:') as tar:
                        tar.extractall(toolkit_dir)
                    tar_path.unlink()
                except:
                    # Резервный способ
                    with gzip.open(archive_path, 'rb') as f_in:
                        tar_path = archive_path.with_suffix('')
                        with open(tar_path, 'wb') as f_out:
                            shutil.copyfileobj(f_in, f_out)
                    with tarfile.open(tar_path, 'r:') as tar:
                        tar.extractall(toolkit_dir)
                    tar_path.unlink()
            
            # Удаляем архив после успешной распаковки
            archive_path.unlink(missing_ok=True)
            
            if self.check_toolkit_extracted(toolkit_dir):
                if self.verbose:
                    print(f"    ✓ Toolkit успешно установлен")
                return True
            else:
                print(f"    ✗ Ошибка: toolkit не распакован корректно")
                return False
                
        except Exception as e:
            print(f"    ✗ Ошибка распаковки: {e}")
            return False
    
    def get_existing_files_report(self):
        """Отчет о существующих файлах."""
        print(f"\n{'='*60}")
        print(f"ПРОВЕРКА СУЩЕСТВУЮЩИХ ФАЙЛОВ")
        print(f"{'='*60}")
        
        categories = {
            'Эфемериды': self.base_dir / 'kernels' / 'spk',
            'Файлы времени': self.base_dir / 'kernels' / 'lsk',
            'Планетные константы': self.base_dir / 'kernels' / 'pck',
            'SPICE Toolkits': self.base_dir / 'toolkits',
        }
        
        for category, path in categories.items():
            if path.exists():
                files = list(path.glob('*'))
                if files:
                    print(f"\n{category}:")
                    for file in files:
                        if file.is_file():
                            size = file.stat().st_size
                            size_str = f"{size/1024/1024:.1f} MB" if size > 1000000 else f"{size/1024:.0f} KB"
                            print(f"  ✓ {file.name} ({size_str})")
        
        print(f"{'='*60}")
    
    def download_all_smart(self, ephemeris_version='de405'):
        """
        Загрузка всех необходимых данных.
        
        Args:
            ephemeris_version: Версия эфемерид для загрузки ('de405', 'de430', 'de440')
        """
        print(f"\n{'='*60}")
        print(f"ЗАГРУЗКА ДАННЫХ ДЛЯ ОБРАБОТКИ ДОПЛЕРА MGS")
        print(f"{'='*60}")
        print(f"Базовая директория: {self.base_dir.absolute()}")
        print(f"Версия эфемерид: {ephemeris_version.upper()}")
        print(f"Принудительная перезапись: {'ВКЛ' if self.force_redownload else 'ВЫКЛ'}")
        print(f"{'='*60}")
        
        # Показываем что уже есть
        self.get_existing_files_report()
        
        if self.force_redownload:
            print(f"\n⚠ Внимание: включен режим ПРИНУДИТЕЛЬНОЙ перезаписи файлов!")
        else:
            print(f"\nℹ Режим: пропуск существующих файлов")
        
        status = {}
        
        # 1. Эфемериды
        print(f"\n1. ПРОВЕРКА ЭФЕМЕРИД {ephemeris_version.upper()}...")
        status['ephemeris'] = self.download_ephemeris_smart(ephemeris_version)
        
        # 2. Базовые ядра SPICE
        print(f"\n2. ПРОВЕРКА БАЗОВЫХ ЯДЕР SPICE...")
        status['spice_kernels'] = self.download_spice_kernels_smart()
        
        # 3. SPICE Toolkit
        print(f"\n3. ПРОВЕРКА SPICE TOOLKIT...")
        status['spice_toolkit'] = self.download_spice_toolkit_smart('PC_Linux_GCC_64bit')
        
        # Итоговый отчет
        print(f"\n{'='*60}")
        print(f"ИТОГ ЗАГРУЗКИ:")
        
        all_downloaded = True
        all_existed = True
        
        for name, success in status.items():
            if success:
                symbol = "✓"
            else:
                symbol = "✗"
                all_downloaded = False
            
            # Проверяем, был ли файл уже загружен ранее
            existed = False
            if name == 'ephemeris':
                ephem_file = self.base_dir / 'kernels' / 'spk' / f"{ephemeris_version}.bsp"
                existed = ephem_file.exists() and not self.force_redownload
            elif name == 'spice_kernels':
                lsk_file = self.base_dir / 'kernels' / 'lsk' / 'naif0012.tls'
                existed = lsk_file.exists() and not self.force_redownload
            
            if existed:
                status_text = "(уже был загружен)"
                all_existed = False
            else:
                status_text = "(скачан только что)"
            
            print(f"  {symbol} {name:20} {status_text}")
        
        print(f"{'='*60}")
        
        # Рекомендации
        if all_downloaded:
            print(f"\nВСЕ ФАЙЛЫ ГОТОВЫ К РАБОТЕ!")
            print(f"Можете запускать решение прямой задачи уточнения орбиты.")
        elif all_existed:
            print(f"\nВСЕ ФАЙЛЫ УЖЕ БЫЛИ ЗАГРУЖЕНЫ РАНЕЕ")
            print(f"Для перезаписи используйте параметр force_redownload=True")
        else:
            print(f"\nНЕКОТОРЫЕ ФАЙЛЫ НЕ ЗАГРУЖЕНЫ")
            print(f"Но основных файлов (эфемериды, базовые ядра) достаточно для работы.")
        
        # Интеграция с вашим парсером
        print(f"\n{'='*60}")
        print(f"ИНТЕГРАЦИЯ С ВАШИМ ПАРСЕРОМ:")
        print(f"{'='*60}")
        
        ephem_path = self.base_dir / 'kernels' / 'spk' / f"{ephemeris_version}.bsp"
        lsk_path = self.base_dir / 'kernels' / 'lsk' / 'naif0012.tls'
        
        
        return status

# ЗАПУСК С РАЗНЫМИ ПАРАМЕТРАМИ
if __name__ == "__main__":
    print("\n" + "="*70)
    print("УМНАЯ ЗАГРУЗКА ДАННЫХ ДЛЯ MGS")
    print("="*70)
    
    # Вариант 1: Обычный режим (пропуск существующих) с DE405
    print("\nВАРИАНТ 1: Режим пропуска существующих файлов (DE405)")
    downloader1 = SmartMGSSpiceDownloader(
        base_dir="mgs_spice_data_de405",
        verbose=True,
        force_redownload=False  # Пропускать существующие файлы
    )
    
    # Запускаем обычный режим с DE405
    choice = input("\nЗапустить загрузку DE405? (y/n): ").strip().lower()
    if choice == 'y':
        downloader1.download_all_smart(ephemeris_version='de405')
    else:
        print("Загрузка отменена.")
    
    print("\n" + "="*70)
    print("СКРИПТ ЗАВЕРШЕН")
    print("="*70)

  0%|          | 32.0k/18.3M [4:04:53<2384:56:40, 2.23B/s]
 60%|█████▉    | 10.9M/18.3M [4:07:18<2:47:44, 770B/s]
 64%|██████▎   | 179M/282M [3:27:43<1:58:19, 15.1kB/s]



УМНАЯ ЗАГРУЗКА ДАННЫХ ДЛЯ MGS

ВАРИАНТ 1: Режим пропуска существующих файлов (DE405)

ЗАГРУЗКА ДАННЫХ ДЛЯ ОБРАБОТКИ ДОПЛЕРА MGS
Базовая директория: /home/ksenia/spacecraft_orbit-clarification-update-parser/mgs_spice_data_de405
Версия эфемерид: DE405
Принудительная перезапись: ВЫКЛ

ПРОВЕРКА СУЩЕСТВУЮЩИХ ФАЙЛОВ

Эфемериды:
  ✓ de405.bsp (62.4 MB)

Файлы времени:
  ✓ naif0012.tls (5 KB)

Планетные константы:
  ✓ pck00010.tpc (123 KB)

SPICE Toolkits:

ℹ Режим: пропуск существующих файлов

1. ПРОВЕРКА ЭФЕМЕРИД DE405...
    ✓ Файл уже существует: de405.bsp (62.4 MB)

2. ПРОВЕРКА БАЗОВЫХ ЯДЕР SPICE...

Проверка базовых ядер SPICE...
    ✓ Файл уже существует: naif0012.tls (0.0 MB)
    ✓ Файл уже существует: pck00010.tpc (0.1 MB)

3. ПРОВЕРКА SPICE TOOLKIT...
    Toolkit уже распакован: PC_Linux_GCC_64bit

ИТОГ ЗАГРУЗКИ:
  ✓ ephemeris            (уже был загружен)
  ✓ spice_kernels        (уже был загружен)
  ✓ spice_toolkit        (скачан только что)

ВСЕ ФАЙЛЫ ГОТОВЫ К РАБОТЕ!
Можете запу

Проверка, что все данные установились верно: